In [40]:
# importing all the required packages

import pandas as pd
import numpy as np
import pandas_datareader as reader
import datetime as dt
import yfinance as yf
from pandas.tseries.offsets import MonthEnd

In [41]:
# Get the tickers of DAX index from Wiki

dax=pd.read_html("https://en.wikipedia.org/wiki/DAX")[4]

In [42]:
tickers= dax.Ticker.to_list()
tickers

['ADS.DE',
 'AIR.DE',
 'ALV.DE',
 'BAS.DE',
 'BAYN.DE',
 'BEI.DE',
 'BMW.DE',
 'BNR.DE',
 'CON.DE',
 '1COV.DE',
 'DTG.DE',
 'DBK.DE',
 'DB1.DE',
 'DPW.DE',
 'DTE.DE',
 'EOAN.DE',
 'FRE.DE',
 'FME.DE',
 'HNR1.DE',
 'HEI.DE',
 'HEN3.DE',
 'IFX.DE',
 'LIN.DE',
 'MBG.DE',
 'MRK.DE',
 'MTX.DE',
 'MUV2.DE',
 'PAH3.DE',
 'PUM.DE',
 'QIA.DE',
 'RWE.DE',
 'SAP.DE',
 'SRT3.DE',
 'SIE.DE',
 'ENR.DE',
 'SHL.DE',
 'SY1.DE',
 'VOW3.DE',
 'VNA.DE',
 'ZAL.DE']

In [43]:
# Getting the price data for the components of DAX index using yfinance

dax_price=yf.download(tickers, start="2009-12-31", end="2021-12-31")

[*********************100%***********************]  40 of 40 completed


In [44]:
prices=dax_price["Adj Close"]
prices.head(3)

,1COV.DE,ADS.DE,AIR.DE,ALV.DE,BAS.DE,BAYN.DE,BEI.DE,BMW.DE,BNR.DE,CON.DE,...,QIA.DE,RWE.DE,SAP.DE,SHL.DE,SIE.DE,SRT3.DE,SY1.DE,VNA.DE,VOW3.DE,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,31.165903,11.332298,47.995468,24.106215,36.843716,39.553482,19.280518,NaN,27.060913,...,15.724052,39.131073,27.136261,NaN,41.060322,3.695453,12.404505,NaN,46.819508,NaN
2010-01-05,NaN,32.149315,11.251352,48.141823,23.740726,36.170856,39.344837,19.436930,NaN,28.753778,...,15.484903,38.881393,26.893240,NaN,41.085411,3.573745,12.408502,NaN,45.912148,NaN
2010-01-06,NaN,31.890308,11.328251,48.515854,23.891220,35.942219,39.315029,19.737719,NaN,28.941877,...,15.709105,38.807625,27.371046,NaN,41.235935,3.606939,12.808260,NaN,47.182446,NaN


In [45]:
# Replacing the dots with dashes in the ticker symbol

ticker=[]

for i in tickers:
    ticker.append(i.replace(".","-"))

In [46]:
# changing the column names in the prices df

prices.columns=ticker

In [47]:
# calculating monthly returns from the daily prices. This means calculating monthly returns by aggregating all the daily returns
# at month t and doing the same for all the following months. And finally, calculating pct_change from month t to month t+1

# And note that the daily returns have to be compunded to finally get to the end of the month returns. 

mtl_rets=prices.pct_change().resample("M").agg(lambda x:(x+1).prod()-1)
mtl_rets.head(3)

,ADS-DE,AIR-DE,ALV-DE,BAS-DE,BAYN-DE,BEI-DE,BMW-DE,BNR-DE,CON-DE,1COV-DE,...,RWE-DE,SAP-DE,SRT3-DE,SIE-DE,ENR-DE,SHL-DE,SY1-DE,VOW3-DE,VNA-DE,ZAL-DE
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-31,0.0,-0.041423,0.016429,-0.091710,-0.080825,-0.122784,-0.089784,-0.034009,0.000000,0.084958,...,0.000634,-0.068446,0.004250,0.0,0.018028,0.012575,0.039317,0.0,-0.091473,0.0
2010-02-28,0.0,-0.014088,0.066058,0.054464,0.000364,-0.016271,0.065050,-0.037952,0.000000,-0.157375,...,0.015199,-0.030511,-0.009674,0.0,-0.022035,0.026020,-0.029457,0.0,0.020478,0.0
2010-03-31,0.0,0.088211,-0.015821,0.094694,0.113482,0.028971,-0.016435,0.147558,0.018518,0.113929,...,0.063631,0.053308,0.094628,0.0,0.177171,0.132565,0.126517,0.0,0.142281,0.0


In [48]:
mtl_rets.shape

(144, 40)

In [49]:
# Calculating rolling 11 months returns (compounded ofcourse)

rolling_rets=(mtl_rets+1).rolling(11).apply(np.prod)-1 # np.prod helps in compounding the monthly returns

In [50]:
# defining the formation date 
# formation date = date on which we gonna buy the winners and short sell the losers

formation_date= dt.datetime(2011,1,1)
formation_date

datetime.datetime(2011, 1, 1, 0, 0)

In [51]:
# defining the end date
# end date= The date up until which the prior monthly returns are calculated and on the basis of this "end_date", we are gonna 
            # sort the winners and losers on the formation date. 
            # In the case of momentum, the end_date includes past 11 months of prior returns (compounded ofcourse)
            # 11 months because the evidence suggests that the short term reversals in momentum is a real and strong phenomenom
            
            
end_date=formation_date - MonthEnd(1)
end_date

Timestamp('2010-12-31 00:00:00')

In [52]:
# locate the end_date returns in ret12_1 df

ret12_1=rolling_rets.loc[end_date]

In [53]:
#reseting the index so that it is converted into a df

ret12_1=ret12_1.reset_index()

In [54]:
ret12_1.head()

,index,2010-12-31 00:00:00
0,ADS-DE,0.000000
1,AIR-DE,0.335663
2,ALV-DE,0.268447
3,BAS-DE,0.162344
4,BAYN-DE,0.503817


In [55]:
type(ret12_1)

pandas.core.frame.DataFrame

In [56]:
# Dividing the above tickers into Terciles on the basis of their cumulative returns upto the end_date

ret12_1["tercile"]=pd.qcut(ret12_1.iloc[:,1], 3, labels=False, duplicates="drop")
ret12_1.head(3)

,index,2010-12-31 00:00:00,tercile
0,ADS-DE,0.000000,0
1,AIR-DE,0.335663,2
2,ALV-DE,0.268447,1


In [57]:
# Lets now assign tercile 2 as winners and tercile 0 as losers

winners=ret12_1[ret12_1.tercile==2]
losers=ret12_1[ret12_1.tercile==0]


In [58]:
winners.head(3)

,index,2010-12-31 00:00:00,tercile
1,AIR-DE,0.335663,2
4,BAYN-DE,0.503817,2
7,BNR-DE,0.915332,2


In [59]:
# So, we are now at the formation date. This means after knowing which ticker belongs to which tercile, now its time to 
# invest and hold the stock for 1 month and see how the return looks like

win_ret=mtl_rets.loc[formation_date + MonthEnd(1), mtl_rets.columns.isin(winners["index"])]
win_ret.head(3)

AIR-DE    -0.069544
BAYN-DE   -0.058961
BNR-DE    -0.047069
Name: 2011-01-31 00:00:00, dtype: float64

In [60]:
los_ret=mtl_rets.loc[formation_date + MonthEnd(1), mtl_rets.columns.isin(losers["index"])]
los_ret.head(3)

ADS-DE    0.000000
BMW-DE   -0.036243
DBK-DE    0.104092
Name: 2011-01-31 00:00:00, dtype: float64

In [61]:
win_ret.mean()

0.002349338106174983

### Now, lets turn all of the above into a function

In [62]:
def momentum(formation_date): # form date in the brackers coz thats what we gonna use to call the function
    end_date=formation_date - MonthEnd(1) # defining end
    ret12_1=rolling_rets.loc[end_date] # locating the end date returns in rolling_rets df
    ret12_1=ret12_1.reset_index() # reseting the index of ret12_1 to make it a df
    ret12_1["tercile"]=pd.qcut(ret12_1.iloc[:,1], 3, labels=False, duplicates="drop") # dividing the returns in ret12_1 into terciles
    winners=ret12_1[ret12_1.tercile==2] # just separating winners and losers
    losers=ret12_1[ret12_1.tercile==0]
    win_rets=mtl_rets.loc[formation_date + MonthEnd(1), mtl_rets.columns.isin(winners["index"])]# going long on the winners at the formation date and holding for 1 month 
    los_rets=mtl_rets.loc[formation_date + MonthEnd(1), mtl_rets.columns.isin(losers["index"])]
    momL_profit=win_rets.mean()
    
    return momL_profit

In [63]:
momentum(formation_date)

0.002349338106174983

#### Lets write a for loop so that we can interate the above function on all the other months

In [64]:
mtl_rets.tail()

,ADS-DE,AIR-DE,ALV-DE,BAS-DE,BAYN-DE,BEI-DE,BMW-DE,BNR-DE,CON-DE,1COV-DE,...,RWE-DE,SAP-DE,SRT3-DE,SIE-DE,ENR-DE,SHL-DE,SY1-DE,VOW3-DE,VNA-DE,ZAL-DE
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-31,0.011795,-0.018458,-0.001383,-0.052476,-0.010571,-0.062537,0.025961,-0.041845,0.014727,-0.006806,...,0.037176,0.102633,0.054783,0.057830,0.070375,0.094547,-0.029767,0.017444,-0.020438,0.000427
2021-09-30,0.080874,-0.095357,-0.005538,-0.020806,0.005037,-0.002970,-0.090414,0.029738,-0.056882,-0.167194,...,-0.044378,-0.076760,-0.083006,-0.046180,0.007668,-0.014695,-0.057628,-0.092022,-0.038053,-0.154371
2021-10-31,-0.066397,0.042311,-0.037939,0.032386,-0.054214,0.035422,-0.016692,0.054132,0.020601,0.070999,...,0.062514,0.089034,0.072211,0.022784,-0.011556,0.019280,0.051914,0.010790,0.000930,0.028492
2021-11-30,-0.100361,-0.097953,-0.106639,-0.043301,-0.070649,-0.085268,-0.040044,-0.025905,-0.079523,-0.067179,...,0.029628,0.026450,-0.096872,0.119387,0.007129,0.083155,0.041405,-0.065955,-0.165927,-0.016180
2021-12-31,0.087480,-0.009196,0.131113,0.079038,0.067381,0.055705,0.024484,0.041304,0.051255,-0.016790,...,-0.000204,0.045974,0.103552,0.023321,0.080691,-0.019440,0.046586,-0.010204,0.097860,-0.113631


In [76]:
profits=[]
dates=[] ## setup some empty lists for storing up the data later

for i in range(132):
    profits.append(momentum(formation_date + MonthEnd(i)))
    dates.append(formation_date + MonthEnd(i))


In [102]:
# converting lists to df

mom_profit=pd.DataFrame(profits)
mom_dates=pd.DataFrame(dates)

# concatenating the two dfs

comb_df=pd.concat([mom_dates, mom_profit], axis=1)

# changing the column names

comb_df.columns=["Date", "Returns"]



In [103]:
comb_df

,Date,Returns
0,2011-01-31,0.020625
1,2011-01-31,0.020625
2,2011-02-28,-0.005771
3,2011-03-31,0.073325
4,2011-04-30,0.002130
...,...,...
127,2021-07-31,0.012300
128,2021-08-31,-0.031163
129,2021-09-30,0.030444
130,2021-10-31,-0.052321


In [104]:
# Indentifying duplicate row(s)

comb_df.duplicated()

0      False
1       True
2      False
3      False
4      False
       ...  
127    False
128    False
129    False
130    False
131    False
Length: 132, dtype: bool

In [105]:
# Removing the duplicate row while keeping the first row still in the df

comb_df=comb_df.drop_duplicates(subset=None, keep="first", inplace=False)

In [106]:
comb_df

,Date,Returns
0,2011-01-31,0.020625
2,2011-02-28,-0.005771
3,2011-03-31,0.073325
4,2011-04-30,0.002130
5,2011-05-31,0.030790
...,...,...
127,2021-07-31,0.012300
128,2021-08-31,-0.031163
129,2021-09-30,0.030444
130,2021-10-31,-0.052321



#### Comparing the above derived mom profits to the DAX index's returns

In [80]:
dax40=yf.download("^GDAXI", start=dates[0], end=dates[-1])

[*********************100%***********************]  1 of 1 completed


In [81]:
dax40=dax40["Adj Close"]

In [82]:
dax40.head(3)

Date
2011-01-31    7077.479980
2011-02-01    7184.270020
2011-02-02    7183.669922
Name: Adj Close, dtype: float64

In [83]:
# calculating monthly pct_change from the daily returns

dax_mtl=dax40.pct_change().resample("M").agg(lambda x:(x-1).prod()-1)

In [84]:
dax_mtl

Date
2011-01-31    0.000000
2011-02-28   -0.027938
2011-03-31   -2.028858
2011-04-30   -1.934756
2011-05-31    0.027854
                ...   
2021-07-31   -0.003049
2021-08-31   -0.018772
2021-09-30    0.035489
2021-10-31   -1.971431
2021-11-30   -2.024525
Freq: M, Name: Adj Close, Length: 131, dtype: float64

In [85]:
dax_mtl.shape

(131,)

#### Combining dax returns and mom profits into a same df

In [107]:
comb_df["Dax40"]=dax_mtl.values
comb_df.head(3)

C:\Users\user\AppData\Local\Temp\ipykernel_4120\3757022027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb_df["Dax40"]=dax_mtl.values


,Date,Returns,Dax40
0,2011-01-31,0.020625,0.000000
2,2011-02-28,-0.005771,-0.027938
3,2011-03-31,0.073325,-2.028858


In [108]:
comb_df["Excess_Returns"]=comb_df["Returns"] - comb_df["Dax40"]

C:\Users\user\AppData\Local\Temp\ipykernel_4120\542529438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb_df["Excess_Returns"]=comb_df["Returns"] - comb_df["Dax40"]


In [109]:
comb_df.head(3)

,Date,Returns,Dax40,Excess_Returns
0,2011-01-31,0.020625,0.000000,0.020625
2,2011-02-28,-0.005771,-0.027938,0.022166
3,2011-03-31,0.073325,-2.028858,2.102183


In [110]:
# Outperformed or not ?? Using list comprehension

comb_df["Outperformed"]=["Yes" if i > 0 else "No" for i in comb_df.Excess_Returns]

C:\Users\user\AppData\Local\Temp\ipykernel_4120\3931131111.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb_df["Outperformed"]=["Yes" if i > 0 else "No" for i in comb_df.Excess_Returns]


In [111]:
comb_df.head(3)

,Date,Returns,Dax40,Excess_Returns,Outperformed
0,2011-01-31,0.020625,0.000000,0.020625,Yes
2,2011-02-28,-0.005771,-0.027938,0.022166,Yes
3,2011-03-31,0.073325,-2.028858,2.102183,Yes


In [113]:
# Hit rate ?? Yes/Total

comb_df[comb_df.Outperformed=="Yes"].shape

(108, 5)

In [115]:
comb_df.shape

(131, 5)

In [116]:
108/131

0.8244274809160306

#### Hit rate of 82.44 % is extremely good. There must be something wrong with my code :D